In [23]:
using NCDatasets
using Plots, ColorSchemes, LaTeXStrings
using DataFrames, CSV
using SparseArrays, LinearAlgebra


function rstar_analysis(ds1, ds2, season=nothing)

    N, P, Z, B, D = get_endpoints(["n", "p", "z", "b", "d"], ds1)
    np = get_size([P])[1]

    N2, P2, Z2, B2, D2 = get_endpoints(["n", "p", "z", "b", "d"], ds2)
    np2 = get_size([P2])[1]

    rs1 = get_rstar_B(B, Z, np, ds1, season)
    rs2 = get_rstar_B_1(B2, Z2, np2, ds2, "Win")
    # RstarP = get_rstar_P(P, Z, ds, season)

    return rs1, rs2

end 



#-----------------------------------------------------------------------------------
#                                     RSTAR B 
#-----------------------------------------------------------------------------------
function get_rstar_B(B, Z, np, ds, season=nothing)

    nb, nz = get_size([B])[1], get_size([Z])[1]
    
    mort_b = b_mortality(B, ds, nb)
    grz_b = b_grazing(B, Z, np, nb, nz, ds)
    loss_b = b_loss(mort_b, grz_b, nb)
    RstarB_ij = RstarB(loss_b, ds, season)

    return RstarB_ij

end

function get_rstar_B_1(B, Z, np, ds, season=nothing)

    nb, nz = get_size([B])[1], get_size([Z])[1]
    
    mort_b = b_mortality(B, ds, nb)
    grz_b = b_grazing_1(B, Z, np, nb, nz, ds)
    loss_b = b_loss_1(mort_b, grz_b, nb)
    RstarB_ij = RstarB_1(loss_b, nb, ds, season)

    return RstarB_ij

end

function b_mortality(B, ds, nb)

    # mort_b = Any[]
    # for i in range(1, nb)
    #     push!(mort_b, (ds["m_lb"][i] .+ ds["m_qb"][i] .* B[:,i]))
    # end
    ngrid = length(B[:,1])
    mort_b = zeros(Float64, ngrid, nb) 
    for i in range(1, nb)
        mort_b[:, i] += (ds["m_lb"][i] .+ ds["m_qb"][i] .* B[:,i])
    end

    return mort_b

end


function b_grazing(B, Z, np, nb, nz, ds)

    GrM = ds["GrM"][:]
    g_max = 1.0
    K_g = 1.0
    ngrid = length(B[:,1])

    grazing = Any[]
    grazing_zi = zeros(Float64, ngrid, nb) 
    for z in range(1, nz)
        if sum(GrM[z,np+1:end]) > 0 
            prey = GrM[z,np+1:end]' .*B[:,1:end]
            g = g_max .* prey ./ (prey .+ K_g)
            grazing_zi += (g .* Z[:,z] .* GrM[z,np+1:end]') ./ prey
            grazing_zi = replace!(grazing_zi, NaN => 0.0)
            push!(grazing, grazing_zi)
        end   
    end
    return sum(grazing)
end

function b_grazing_1(B, Z, np, nb, nz, ds)
    #NOTE this all needs to be generalized / scaled !! 
    GrM = ds["GrM"][:]
    g_max, K_g = 1.0, 1.0
    ngrid = length(B[:,1])

    grazing = Any[]
    # ------- for 1N 8P 6Z 13B 5D
    if nb == 13
        # NOTE length(B[:,1]) = prms.ngrid
        grazing_zi = zeros(Float64, ngrid, nb) 
        for z in range(1, nz)
            if sum(GrM[z,np+1:end]) > 0 
                prey = GrM[z,np+1:end]' .*B[:,1:end]
                g = g_max .* prey ./ (prey .+ K_g)
                grazing_zi += (g .* Z[:,z] .* GrM[z,np+1:end]') ./ prey
                grazing_zi = replace!(grazing_zi, NaN => 0.0)
                push!(grazing, grazing_zi)
            end   
        end
        return sum(grazing)
    end

    #------- for 1N 4P 3Z 7B 4D
    if nb==7
        prey_POM = GrM[2,5]' .* B[:,1]
        gb_POM = g_max .* prey_POM ./ (prey_POM .+ K_g)
        grz_POM = (gb_POM .* Z[:,2] .* GrM[2,5]' ./ prey_POM) 
        push!(grazing, grz_POM)

        prey = GrM[3,6:end]' .* B[:,2:end]
        for i in range(1, nb-1)
            gb_i = g_max .* prey[:,i] ./ (prey[:,i] .+ K_g)
            grz_i = (gb_i .* Z[:,3] .* GrM[3,i+(np+1)]' ./ prey[:,i]) 

            if any(isnan.(grz_i)) 
                return true
            end

            push!(grazing, grz_i)
        end
    
        return grazing
    
    else

        #----- for 1N 2P 2Z 2B 2D
        if nz == 2
            K_g = ds["K_g"][2]
            prey = GrM[2,3:end]' .*B[:,1:end]

            for i in range(1, nb)
                gb_i = g_max .* prey[:,i] ./ (prey[:,i] .+ K_g)
                grz_i = (gb_i .* Z[:,2] .* GrM[2,i+2]' ./ prey[:,i])
                push!(grazing, grz_i)
            end

        #----- for 1N 1P 3Z 2B 2D
        elseif nz == 3
            K_g = [ds["K_g"][2], ds["K_g"][3]]
            prey = 1.0 .*B[:,1:2]

            for i in range(1, nb)
                gb_i = g_max .* prey[:,i] ./ (prey[:,i] .+ K_g[i])
                grz_i = (gb_i .* Z[:,i+1] .* 1.0 ./ prey[:,i]) 
                push!(grazing, grz_i)
            end
        else
        end
    end

    return grazing

end


function b_loss(mortality, grazing, nb)

    loss = zeros(Float64, 89, nb)
    for i in range(1, nb)
        loss[:, i] = mortality[:, i] .+ grazing[:, i]
    end

    return loss
end

function b_loss_1(mortality, grazing, nb)

    if nb == 13
        loss = zeros(Float64, 89, nb)
        for i in range(1, nb)
            loss[:, i] = mortality[:, i] .+ grazing[:, i]
        end
    else
        loss = Any[]
        for i in range(1, nb)
            push!(loss, mortality[i] .+ grazing[i])
        end
    end
    return loss
end



function RstarB(loss, ds, season)

    season !== nothing ? temp_mod = get_temp_mod(season) : temp_mod = ds["temp_fun"][:]

    vmax_ij = ds["vmax_ij"][:]
    Km_ij = ds["Km_ij"][:]
    yield = ds["y_ij"][:]
    II, JJ = get_nonzero_axes(ds["CM"][:])
    RS = Any[]

    for j = axes(II, 1)
        push!(RS, Km_ij[II[j],JJ[j]] .* loss[:, j] ./ (yield[II[j],JJ[j]] .* vmax_ij[II[j],JJ[j]] .* temp_mod .- loss[:, j]))
    end

    for i in range(1, length(RS))
        RS[i] = check_for_negatives(RS[i])
    end

    return RS

end

function RstarB_1(loss, nb, ds, season)

    season !== nothing ? temp_mod = get_temp_mod(season) : temp_mod = ds["temp_fun"][:]

    vmax_ij = ds["vmax_ij"][:]
    Km_ij = ds["Km_ij"][:]
    yield = ds["y_ij"][:]
    II, JJ = get_nonzero_axes(ds["CM"][:])
    RS = Any[]

    if nb == 13
        for j = axes(II, 1)
            push!(RS, Km_ij[II[j],JJ[j]] .* loss[:, j] ./ (yield[II[j],JJ[j]] .* vmax_ij[II[j],JJ[j]] .* temp_mod .- loss[:, j]))
        end
    else
        for j = axes(II, 1)
            push!(RS, Km_ij[II[j],JJ[j]] .* loss[j] ./ (yield[II[j],JJ[j]] .* vmax_ij[II[j],JJ[j]] .* temp_mod .- loss[j]))
        end
    end

    for i in range(1, length(RS))
        RS[i] = check_for_negatives(RS[i])
    end

    return RS

end

function get_temp_mod(season)
    #fit to SPOT data (approx 20 to 4, approx 16 to 4)
    if season == "Win"
        temp_mod = CSV.read("/home/lee/Dropbox/Development/NPZBD_1D/data/temp_mod/win_temp_mod.csv", DataFrame)
    else
        temp_mod = CSV.read("/home/lee/Dropbox/Development/NPZBD_1D/data/temp_mod/sum_temp_mod.csv", DataFrame)
    end

    return Matrix(temp_mod)
end


function check_for_negatives(RS)

    for i in eachindex(RS)
        RS[i] = ifelse(RS[i] < 0, NaN, RS[i])
        RS[i] = ifelse(RS[i] > 15, NaN, RS[i])
    end

    return RS

end


function get_endpoints(vars, ds=nothing)

    endpoints = Vector{Any}()

    if ds !== nothing
        for v in vars
            append!(endpoints, [ds["$v"][:,:,end]])
        end
    else
        for v in vars
            append!(endpoints, [v[:,:,end]])
        end
    end

    return endpoints
end

function get_size(arr)

    out = Vector{Int}()
    
    for a in arr
        append!(out, size(a, 2))
    end

    return out

end

function get_nonzero_axes(Mat)

    Cs = sparse(Mat)
    (II, JJ, _) = findnz(Cs) 
    
    return II, JJ

end 

get_nonzero_axes (generic function with 1 method)

In [ ]:
ds = NCDataset("/home/lee/Dropbox/Development/NPZBD_1D/results/outfiles/endpoints/Wi100y_230923_17:23_8P6Z13B5D_ep.nc")
ds2 = NCDataset("/home/lee/Dropbox/Development/NPZBD_1D/results/outfiles/endpoints/Wi100y_230905_20:05_2P2Z2B2D_ep.nc")

In [35]:
N, P, Z, B, D = get_endpoints(["n", "p", "z", "b", "d"], ds)
np = get_size([P])[1]
nb = get_size([B])[1]
nz = get_size([Z])[1]

6

In [36]:
mb = b_mortality(B, ds, nb)

89×13 Matrix{Float64}:
 0.0636155  0.01  0.0121454  0.0203455  …  0.01  0.01  0.01  0.01  0.01  0.01
 0.0635482  0.01  0.0121429  0.0203319     0.01  0.01  0.01  0.01  0.01  0.01
 0.0633756  0.01  0.0121366  0.0202963     0.01  0.01  0.01  0.01  0.01  0.01
 0.0630438  0.01  0.0121246  0.0202269     0.01  0.01  0.01  0.01  0.01  0.01
 0.0624785  0.01  0.0121045  0.0201072     0.01  0.01  0.01  0.01  0.01  0.01
 0.0615822  0.01  0.0120732  0.0199158  …  0.01  0.01  0.01  0.01  0.01  0.01
 0.0602372  0.01  0.012027   0.0196274     0.01  0.01  0.01  0.01  0.01  0.01
 0.0583192  0.01  0.0119618  0.0192161     0.01  0.01  0.01  0.01  0.01  0.01
 0.0557251  0.01  0.011874   0.0186626     0.01  0.01  0.01  0.01  0.01  0.01
 0.052414   0.01  0.0117609  0.0179624     0.01  0.01  0.01  0.01  0.01  0.01
 ⋮                                      ⋱                    ⋮           
 0.0105745  0.01  0.0100018  0.0100019  …  0.01  0.01  0.01  0.01  0.01  0.01
 0.0105783  0.01  0.0100018  0.0100019     0.

In [37]:
gb = b_grazing(B, Z, np, nb, nz, ds)
gb2 = b_grazing_1(B, Z, np, nb, nz, ds)

gb == gb2

true

In [38]:
lb = b_loss(mb, gb, nb)
lb2 = b_loss_1(mb, gb, nb)

lb == lb2

true

In [39]:
rs = RstarB(lb, ds, season=nothing)
rs2= RstarB_1(lb, nb, ds, "Win")

MethodError: MethodError: no method matching RstarB(::Matrix{Float64}, ::NCDataset{Nothing}; season=nothing)
Closest candidates are:
  RstarB(::Any, ::Any, !Matched::Any) at ~/Dropbox/Development/NPZBD_1D/notebooks/test.ipynb:196 got unsupported keyword argument "season"